Comenzamos cargandos las librerias y los datasets a analizar

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_mt = pd.read_csv("../data/processed/games_metadata.csv")
df_games = pd.read_csv("../data/processed/games.csv")
df_recomendations = pd.read_csv("../data/processed/recommendations.csv")
df_users = pd.read_csv("../data/processed/users.csv")

Los datos los hemos obtenido de Kaggle, donde le usuario indica que la fuente el STEAMDB, la Base de datos oficial de Steam, por lo que estos datos son los oficiales publicados por la plataforma.

Ahora, revisaremoss las primeras lineas de cada dataset para revisar la información que contiene cada uno.

In [8]:
print("Dataset games_metadata")
df_mt.head()

Dataset games_metadata


,app_id,description,tags
0,13500,Enter the dark underworld of Prince of Persia ...,"Action, Adventure, Parkour, Third Person, Grea..."
1,113020,Monaco: What's Yours Is Mine is a single playe...,"Co-op, Stealth, Indie, Heist, Local Co-Op, Str..."
2,226560,Escape Dead Island is a Survival-Mystery adven...,"Zombies, Adventure, Survival, Action, Third Pe..."
3,249050,Dungeon of the Endless is a Rogue-Like Dungeon...,"Roguelike, Strategy, Tower Defense, Pixel Grap..."
4,250180,"“METAL SLUG 3”, the masterpiece in SNK’s emble...","Arcade, Classic, Action, Co-op, Side Scroller,..."


In [ ]:
print("Dataset games")
df_games.head()

Dataset games


,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


In [ ]:
print("Dataset recommendations")
df_recomendations.head()

Dataset recommendations


,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,True,36.3,51580,0
1,304390,4,0,2017-02-17,False,11.5,2586,1
2,1085660,2,0,2019-11-17,True,336.5,253880,2
3,703080,0,0,2022-09-23,True,27.4,259432,3
4,526870,0,0,2021-01-10,True,7.9,23869,4


In [11]:
print("Dataset users")
df_users.head()

Dataset users


,user_id,products,reviews
0,7360263,359,0
1,14020781,156,1
2,8762579,329,4
3,4820647,176,4
4,5167327,98,2


Gracias a esto, obtenemos la siguiente leyenda de datos de cada columna:

- app_id: (KEY_ID) Código identificativo de cada juego
- description: (STRING) Descripción aportada por la empresa sobre el su juego
- tags: (LIST) Etiquetas añadidas por la empresa para categorizar el tipo de juego
- title: (STRING) Titulo del juego
- date_release: (AAAA-MM-DD) Fecha de lanzamiento del Juego en Steam
- win: (BOOLEAN) Compatibilidad con Windows
- mac: (BOOLEAN) Compatibilidad con MAC
- linux: (BOOLEAN) Compatibilidad con Linux
- rating: (STRING) Idea general de las reseñas generadas por los usuarios sobre el juego
- positive_ratio: (INT) Puntuación media del juego según las reseñas
- user_reviews: (INT) Cantidad total de reseñas sobre el juego
- price_final: (FLOAT) Precio final (actual en el momento de extracción de los datos)
- price_original: (FLOAT) Precio original
- discount: (FLOAT) Descuento (actual en el momento de la extracción de los datos)
- steam_deck: (BOOLEAN) Compatibilidad con Steam Deck
- helpful: (INT) Indica la cantidad de personas que han encontrado la reseña útil
- funny: (INT) Indica la cantidad de personas que han encontrado la reseña divertida
- date: (AAAA-MM-DD) Fecha de la reseña
- is_recommended: (BOOLEAN) Indica si el jugador que publicó la reseña recomienda o no la reseña
- hours: (FLOAT) Cantidad de horas que se ha jugado antes de publicar la reseña
- user_id: (KEY_ID) Id del usuario que publicó al reseña
- review_id: (KEY_ID) Id de la reseña
- products: (INT) Cantidad de juegos en la biblioteca del usuario
- reviews: (INT) Cantidad de reseñas publicadas por el usuario

In [ ]:
# Histograma
sns.histplot(df['edad'])
plt.title("Distribución de edad")
plt.show()

# Correlación
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title("Matriz de correlación")
plt.show()